In [18]:
import os, glob 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
 
fit_path = "/home/riku/projects/athlet_hr_predict/fit_file_csv"

In [27]:
os.chdir(fit_path)


n_X = 60 # number of timesteps for training
n_y = 30 # number of timesteps to predict
step = 1
batch_size = 10
epochs = 10
learning_rate = 0.001
sequence_length = int(n_X/step)
n_fit_files_test_set = 1 # number of files for testing dataset

train_files = glob.glob("*.csv")[0:-n_fit_files_test_set]
valid_files = glob.glob("*.csv")[-n_fit_files_test_set:]

# validation dataset - train each file at the time

for file in valid_files:
    df = pd.read_csv(fit_path+'/'+file, index_col='timestamp')[["heart_rate", "cadence","enhanced_speed","enhanced_altitude"]]
    
    start = n_X + n_y
    end = n_X + len(df.index)
    
    x = df[["heart_rate", "cadence","enhanced_speed","enhanced_altitude"]].values
    y = df.iloc[n_X:end][["heart_rate"]]
    
    dataset_test = keras.preprocessing.timeseries_dataset_from_array(
        x,
        y,
        sequence_length=sequence_length,
        sampling_rate=step,
        batch_size=batch_size,
    )

for batch in dataset_test.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

# training dataset - train each file at the time

for file in train_files:
    df = pd.read_csv(fit_path+'/'+file, index_col='timestamp')[["heart_rate", "cadence","enhanced_speed","enhanced_altitude"]]
    
    start = n_X + n_y
    end = n_X + len(df.index)
    
    x = df[["heart_rate", "cadence","enhanced_speed","enhanced_altitude"]].values
    y = df.iloc[n_X:end][["heart_rate"]]
    
    dataset_train = keras.preprocessing.timeseries_dataset_from_array(
        x,
        y,
        sequence_length=sequence_length,
        sampling_rate=step,
        batch_size=batch_size,
    )






2021-09-14 08:48:56.594124: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Input shape: (10, 60, 4)
Target shape: (10, 1)


In [26]:
dataset_train.batch

<bound method DatasetV2.batch of <BatchDataset shapes: ((None, None, 4), (None, 1)), types: (tf.float64, tf.int64)>>

In [ ]:
 
def last_time_step_mse(Y_true, Y_pred):
  return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model = keras.Sequential([
    keras.layers.LSTM(240, return_sequences=True, input_shape=(60,4)),
    keras.layers.LSTM(60, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(30))   
])

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="mse", optimizer=optimizer, metrics=[last_time_step_mse])